# SSDの損失関数クラスMultiBoxLossの実装

## 目標
1.	jaccard係数を用いたmatch関数の動作を理解する
2.	Hard Negative Miningを理解する
3.	2種類の損失関数（SmoothL1Loss関数、交差エントロピー誤差関数）の働きを理解する


## library

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 8732個のDBoxから正解DBoxと近いDBoxを抽出する関数
from utils.match import match

In [9]:
class MultiBoxLoss(nn.Module):
    def __init__(self, jaccard_thresh=0.5, neg_pos=3, device='cpu'):
        # 親クラスのコンストラクタを実行
        super(self).__init__()
        
        self.jaccard_thresh = jaccard_thresh
        self.negpos_ratio = neg_pos
        self.device = device
        
    def forward(self, predictons, targets):
        """
        損失関数の計算
        
        Parameters
        ---------------
        predictions : SSD netの訓練時の出力(tuple)
            (loc=torch.Size([num_batch, 8732, 4]),
            conf=torch.Size([num_batch, 8732, 21]), 
            dbox_list=torch.Size([8732, 4]))
            
        targets : [num_batch, num_objs, 5]
            5は正解のアノテーション情報[xmin, ymin, xmax, ymax, label_ind]を示す
            
        Returns
        ---------------
        loss_l : Tensor
            locの損失　　smoothL1Loss
        loss_c : Tensor
            confの損失　cross　entropy
        
        """
        
        # SSDのモデルの出力がtupleなのでここにバラす
        loc_data, conf_data, dbox_list = predictions
        
        # 要素数を把握
        num_batch = loc_data.size(0)   # ミニバッチのサイズ
        num_dbox = loc_data.size(1)    # DBoxの数＝8732
        num_classes = conf_data.size(2)  # クラス数＝２１
        
        # 損失の計算に使用するものを格納する変数を作る
        # conf_t_label : 各DBoxに一番近い正解のBBoxのラベルを格納
        # loc_t : 各DBoxに一番近い正解のBBoxの位置情報を格納させる
        conf_t_label = torch.LongTensor(num_batch, num_dbox).to(self.device)  # 64-bit integer
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)# 32-bit floating point
        
        # conf_t_label,　loc_tに
        # DBoxと正解アノテーションtargetsをmatchさせた結果を上書きする
        for idx in range(num_batch):   # ミニバッチでループ
            
            # 現在のミニバッチの正解アノテーションのBBoxトラベルを取得
            truths = targets[idx][:, :, -1].to(self.device)   # BBox
            lables = targets[idx][:,-1].to(self.device)
            
            # デフォルトボックスを新たな変数で用意
            dbox = dbox_list.to(dself.evice)
            
            # match関数を実行しloc_t, conf_t_labelの内容を更新する
            variance = [0.1, 0.2]
            # variance : DBoxからBBoxに補正計算する際に使用する式の係数
            match(self.jaccard_thresh, truths, dbox, variance, labels, loc_t, conf_t_label, idx)
            
        # -------
        # 位置の損失：loss_l を計算
        # SmoothL1Loss  ただし物体を発見したDBoxオフセットのみを計算
        # -------
        # 物体検出したBBoxを取り出すマスクを作成
        pos_mask = conf_t_label > 0  # 背景以外
        # pos_maskをloc_dataのサイズに変形
        pos_idx = pos_mask.unsqeeze(pos_mask.dim()).expand_as(loc_data)
        
        # positive DBoxのloc_dataと教師データloc_tを取得
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        
        # loc_p, loc_tを使って物体を発見したPositive DBoxのオフセット情報loc_tの損失を計算
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')
        
        # -------
        # クラス予測の損失：loss_ｃ を計算
        # Cross Entropy　Loss
        # ただし背景クラスが正解であるDBoxが圧倒的に多いので、
        # Hard Negative Miningを実施し、物体発見DBoxと背景クラスDBoxが1:3の比になるようにする
        # 背景クラスDBoxと予測したもののうち、損失が小さいものはクラス予測の損失から除く
        # -------
        batch_conf = conf_data.view(-1, num_classes)
        
        # クラス予測の損失を関数を使って計算
        # reduction = None　にして和を取らず、次元を潰さない
        loss_c = F.cross_entropy(
            batch_conf, conf_t_label.view(-1), reduction='none')
        
        # -------
        # Hard Negative Mining で抽出するものを求めるマスクを作成
        # -------
        
        # 物体発見したPositive DBoxの損失を０にする
        num_pos = pos_mask.long().sum(1, keepdim=True)  # ミニバッチ毎の物体クラス予測の数
        loss_c = loss_c.view(num_batch, -1)  # torch.Size([num_batch, 8732])
        loss_c[pos_mask] = 0    # 物体を発見したDBoxは損失０とする
        
        # Hard Negative Minigを実施
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rand = loss_idx.sort(1)
        
        num_neg = torch.clamp(num_pos*self.negpos_ratio, max=num_dbox)
        neg_mask = idx_rank < (num_neg).expand_as(idx_rank)
        
        # -------
        # Negative DBoxのうちHarad Negative Miningで抽出するものを求めるマスクを作成
        # -------
        pos_idx_mask = pos_mask.unsqeeze(2).expand_as(conf_data)
        neg_idx_mask = neg_mask.unsqeeze(2).expand_as(conf_data)
        
        conf_hnm = conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1, num_classes)
        
        conf_t_label_hmn = conf_t_label[(pos_mask+neg_mask).gt(0)]
        
        # confidence の損失関数を計算
        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction='sum')
        
        # 物体を発見したBBoxの数N（全ミニバッチの合計）で損失を割り算
        N = num_pos.sum()
        loss_l /= N
        loss_c /= N
        
        return loss_l, loss_c
    